In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlalchemy
import pymysql.cursors
import os
import yaml


## 1. Data Cleaning

### Data Cleaning of the main dataframe (Sorare Cards Transactions)

#### a.  Sorare Transactions Loading & bird's view


In [2]:
all_transactions_df = pd.read_csv('all_players_cards_transactions_la_liga.csv', dtype={20: str})

In [3]:
all_transactions_df.shape

(1741383, 21)

In [4]:
all_transactions_df['player_slug'].nunique()

909

In [5]:
all_transactions_df.dtypes

team_name              object
card_id                object
asset_id               object
card_name              object
season_year             int64
card_slug              object
rarity                 object
card_creation_date     object
grade                   int64
power                 float64
xp                      int64
serial_number           int64
shirt_number            int64
special_edition        object
transaction_date       object
transaction_type       object
price_wei               int64
price_eur             float64
user_id                object
user_slug              object
player_slug            object
dtype: object

In [6]:
all_transactions_df.head()

,team_name,card_id,asset_id,card_name,season_year,card_slug,rarity,card_creation_date,grade,power,...,serial_number,shirt_number,special_edition,transaction_date,transaction_type,price_wei,price_eur,user_id,user_slug,player_slug
0,Real Madrid CF,Card:00035848-b60d-4999-832c-e4dd8e939007,0x040004752c83837b25107500c71efa485aafb6cf1b82...,Nacho 2021-22 • Limited 218/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-218,limited,2021-11-25T03:40:39Z,7,1.045,...,218,6,NaN,2022-01-18T10:23:04Z,REFERRAL,0,0.000000,User:dff07274-2664-4ed9-9308-4de48a2b8bec,serveg,jose-ignacio-fernandez-iglesias
1,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31T08:58:22Z,13,1.075,...,488,6,NaN,2022-07-19T09:23:20Z,ENGLISH_AUCTION,6500000000000000,9.747140,User:ee426172-1955-4317-98d7-480476d0bb28,anonymous-8f8df118-a1d8-493d-9a5e-98997ed6450b,jose-ignacio-fernandez-iglesias
2,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31T08:58:22Z,13,1.075,...,488,6,NaN,2022-07-19T09:36:47Z,SINGLE_SALE_OFFER,5100000000000000,7.647756,User:f831ecc8-990a-42e7-94b5-6656536685f5,bugggyman-ua,jose-ignacio-fernandez-iglesias
3,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31T08:58:22Z,13,1.075,...,488,6,NaN,2022-07-19T11:55:20Z,SINGLE_SALE_OFFER,5400000000000000,8.116470,User:9d7ae8cb-c48e-4e5b-8715-eae0826c06c1,junmisugi84-e6f8dfdd-08b3-463f-a2b5-d5870eb74deb,jose-ignacio-fernandez-iglesias
4,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31T08:58:22Z,13,1.075,...,488,6,NaN,2023-03-14T10:28:55Z,SINGLE_BUY_OFFER,2800000000000000,4.356016,User:2d9f44fb-d504-4159-96bd-89deaa61e585,alvaropm019,jose-ignacio-fernandez-iglesias


#### b. removing null values

In [7]:
all_transactions_df.isna().sum()

team_name                   0
card_id                     0
asset_id                    0
card_name                   0
season_year                 0
card_slug                   0
rarity                      0
card_creation_date          0
grade                       0
power                       0
xp                          0
serial_number               0
shirt_number                0
special_edition       1503306
transaction_date            0
transaction_type            0
price_wei                   0
price_eur                   0
user_id                     0
user_slug                   0
player_slug             61549
dtype: int64

In [8]:
#1 for "special edition" replacing values: 
## - all_transactions_df['special_edition'].value_counts() shows that "classic" edition appears recently and is not a "special edition" attribute. 
## - handled by putting "classic" and none values to "False" and others to "True" and convert to boolean. 


all_transactions_df['special_edition'] = all_transactions_df['special_edition'].fillna('unknown')

# Replace 'classic' with 'False' and all other values with 'True'
all_transactions_df['special_edition'] = all_transactions_df['special_edition'].replace({'classic': False})
all_transactions_df['special_edition'] = all_transactions_df['special_edition'].replace({'unknown': False}, regex=True)
all_transactions_df['special_edition'] = all_transactions_df['special_edition'].replace({'.*': True}, regex=True)

# Convert the column to boolean type
all_transactions_df['special_edition'] = all_transactions_df['special_edition'].astype(bool)

In [9]:
all_transactions_df['special_edition'].value_counts()

special_edition
False    1693790
True       47593
Name: count, dtype: int64

In [10]:
# removing the null values in "player_slug"
all_transactions_df.dropna(subset=['player_slug'], inplace=True)

#### c. Parsing and computing the dates: 

In [11]:
def time_computation(df, column):
    df[column] = pd.to_datetime(df[column])
    df[f'{column}_year'] = df[column].dt.year
    df[f'{column}_quarter'] = df[column].dt.quarter
    df[f'{column}_month'] = df[column].dt.month
    df[f'{column}_week'] = df[column].dt.isocalendar().week
    df[f'{column}_dayofyear'] = df[column].dt.dayofyear
    df[f'{column}_season'] = np.where(df[column].dt.month >= 8, df[f'{column}_year'], df[f'{column}_year'] - 1)
    df[f'{column}_season'] = 'season_' + df[f'{column}_season'].astype(str)
    return df

all_transactions_df = time_computation(all_transactions_df, 'card_creation_date')
all_transactions_df = time_computation(all_transactions_df, 'transaction_date')

In [12]:
all_transactions_df.head()

,team_name,card_id,asset_id,card_name,season_year,card_slug,rarity,card_creation_date,grade,power,...,card_creation_date_month,card_creation_date_week,card_creation_date_dayofyear,card_creation_date_season,transaction_date_year,transaction_date_quarter,transaction_date_month,transaction_date_week,transaction_date_dayofyear,transaction_date_season
0,Real Madrid CF,Card:00035848-b60d-4999-832c-e4dd8e939007,0x040004752c83837b25107500c71efa485aafb6cf1b82...,Nacho 2021-22 • Limited 218/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-218,limited,2021-11-25 03:40:39+00:00,7,1.045,...,11,47,329,season_2021,2022,1,1,3,18,season_2021
1,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31 08:58:22+00:00,13,1.075,...,5,22,151,season_2021,2022,3,7,29,200,season_2021
2,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31 08:58:22+00:00,13,1.075,...,5,22,151,season_2021,2022,3,7,29,200,season_2021
3,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31 08:58:22+00:00,13,1.075,...,5,22,151,season_2021,2022,3,7,29,200,season_2021
4,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31 08:58:22+00:00,13,1.075,...,5,22,151,season_2021,2023,1,3,11,73,season_2022


#### d. Removing transactions when players were not in a la liga clubs

In [13]:
# Removing special cards that don't are interesting in the 

In [14]:
# list of slugs and names of la liga clubs since 2021: 

list_of_team_slugs = ["real-madrid-madrid",
                 "girona-girona",
                 "barcelona-barcelona", 
                 "atletico-madrid-madrid",
                 "athletic-club-bilbao",
                 "real-sociedad-donostia-san-sebastian", 
                 "valencia-valencia", 
                 "real-betis-sevilla",
                 "las-palmas-las-palmas-de-gran-canaria",
                "getafe-getafe-madrid",
                 "deportivo-alaves-vitoria-gasteiz",
                 "osasuna-pamplona-irunea",
                 "rayo-vallecano-madrid",
                 "villarreal-villarreal",
                 "sevilla-sevilla-1890",
                 "celta-de-vigo-vigo",
                 "mallorca-palma-de-mallorca",
                 "cadiz-cadiz",
                 "granada-granada",
                 "almeria-almeria",
                 "real-valladolid-valladolid",
                 "espanyol-barcelona",
                 "elche-elche",
                 "levante-valencia"
                ]

list_of_team_names = ['Real Madrid CF',
                 'Girona FC',
                 'FC Barcelona',
                 'Club Atlético de Madrid',
                 'Athletic Club Bilbao',
                 'Real Sociedad de Fútbol',
                 'Valencia CF',
                 'Real Betis Balompié',
                 'UD Las Palmas',
                 'Getafe Club de Fútbol',
                 'Deportivo Alavés',
                 'CA Osasuna',
                 'Rayo Vallecano',
                 'Villarreal CF',
                 'Sevilla FC',
                 'Real Club Celta de Vigo',
                 'Real Club Deportivo Mallorca',
                 'Cádiz CF',
                 'Granada CF',
                 'UD Almería',
                 'Real Valladolid Club de Fútbol',
                 'Reial Club Deportiu Espanyol',
                 'Elche CF',
                 'Levante UD']

In [15]:
#loading the clubs history of players

players_club_hist_df = pd.read_csv('clubs_hist.csv')

In [16]:
players_club_hist_df.dtypes

player_slug            object
position               object
age                     int64
u23_eligible             bool
u23_until              object
club_slug              object
contract_start_date    object
contract_end_date      object
dtype: object

In [17]:
#parsing the date: 
players_club_hist_df['contract_start_date'] = pd.to_datetime(players_club_hist_df['contract_start_date'])
players_club_hist_df['contract_end_date'] = pd.to_datetime(players_club_hist_df['contract_end_date'])

In [18]:


# Step 1: Filter players_club_hist_df for La Liga club history
la_liga_clubs_df = players_club_hist_df[players_club_hist_df['club_slug'].isin(list_of_team_slugs)]

# Step 2: Merge all_transactions_df with la_liga_clubs_df based on player_slug and contract dates
merged_df = pd.merge(all_transactions_df, la_liga_clubs_df, on='player_slug', how='inner')
filtered_transactions_df = merged_df[
    (merged_df['transaction_date'] >= merged_df['contract_start_date']) &
    (merged_df['transaction_date'] <= merged_df['contract_end_date'])
].copy() # Create a copy of the DataFrame

# Step 3: Rename columns to avoid confusion
filtered_transactions_df.rename(columns={'club_slug': 'club_slug_upon_transaction', 'team_name': 'present_team_name'}, inplace=True)

# Step 4: Drop unnecessary columns
filtered_transactions_df = filtered_transactions_df.drop(columns=['contract_start_date', 'contract_end_date'])



In [19]:
filtered_transactions_df.shape

(1349824, 38)

In [20]:
filtered_transactions_df['club_slug_upon_transaction'].value_counts()

club_slug_upon_transaction
real-madrid-madrid                       89095
sevilla-sevilla-1890                     80973
real-betis-sevilla                       79597
atletico-madrid-madrid                   74357
villarreal-villarreal                    74219
barcelona-barcelona                      67493
athletic-club-bilbao                     66256
osasuna-pamplona-irunea                  63721
mallorca-palma-de-mallorca               60937
real-sociedad-donostia-san-sebastian     59829
celta-de-vigo-vigo                       59657
rayo-vallecano-madrid                    59327
valencia-valencia                        57020
getafe-getafe-madrid                     55180
espanyol-barcelona                       54783
deportivo-alaves-vitoria-gasteiz         53073
granada-granada                          49234
cadiz-cadiz                              47982
levante-valencia                         46221
elche-elche                              44190
girona-girona                    

In [21]:
""" tests to check if it works well. VALIDATED all_rulli_transactions = all_transactions_df[all_transactions_df["player_slug"] == "geronimo-rulli"]
all_rulli_transactions.shape

filt_rulli_transactions = filtered_transactions_df[filtered_transactions_df["player_slug"] == "geronimo-rulli"]
filt_rulli_transactions.shape

all_jude_transactions = all_transactions_df[all_transactions_df["player_slug"] == "jude-bellingham"]
all_jude_transactions.shape

filt_jude_transactions = filtered_transactions_df[filtered_transactions_df["player_slug"] == "jude-bellingham"]
filt_jude_transactions.shape
"""

' tests to check if it works well. VALIDATED all_rulli_transactions = all_transactions_df[all_transactions_df["player_slug"] == "geronimo-rulli"]\nall_rulli_transactions.shape\n\nfilt_rulli_transactions = filtered_transactions_df[filtered_transactions_df["player_slug"] == "geronimo-rulli"]\nfilt_rulli_transactions.shape\n\nall_jude_transactions = all_transactions_df[all_transactions_df["player_slug"] == "jude-bellingham"]\nall_jude_transactions.shape\n\nfilt_jude_transactions = filtered_transactions_df[filtered_transactions_df["player_slug"] == "jude-bellingham"]\nfilt_jude_transactions.shape\n'

#### c. Deal with club_names different input during fetching (replaces slugs by names)

In [22]:
replacement_dict = dict(zip(list_of_team_slugs, list_of_team_names))

filtered_transactions_df['present_team_name'] = filtered_transactions_df['present_team_name'].replace(replacement_dict)

#### d. Filter only on transactions where promoted or relegated were in La Liga

In [27]:
clubs_df = pd.DataFrame({'team_name': list_of_team_names, 'team_slug': list_of_team_slugs})
clubs_df

,team_name,team_slug
0,Real Madrid CF,real-madrid-madrid
1,Girona FC,girona-girona
2,FC Barcelona,barcelona-barcelona
3,Club Atlético de Madrid,atletico-madrid-madrid
4,Athletic Club Bilbao,athletic-club-bilbao
5,Real Sociedad de Fútbol,real-sociedad-donostia-san-sebastian
6,Valencia CF,valencia-valencia
7,Real Betis Balompié,real-betis-sevilla
8,UD Las Palmas,las-palmas-las-palmas-de-gran-canaria
9,Getafe Club de Fútbol,getafe-getafe-madrid


In [28]:
##clubs_df.to_csv("clubs_by_season.csv", index=False)
clubs_by_season_df = pd.read_csv('clubs_by_season.csv', sep=';')
clubs_by_season_df 

,team_name,team_slug,2021_22_la_liga,2022_23_la_liga,2023_24_la_liga,2021_22_uefa,2022_23_uefa,2023_24_uefa
0,Real Madrid CF,real-madrid-madrid,True,True,True,True,True,True
1,Girona FC,girona-girona,False,True,True,False,False,False
2,FC Barcelona,barcelona-barcelona,True,True,True,True,True,True
3,Club Atlético de Madrid,atletico-madrid-madrid,True,True,True,True,True,True
4,Athletic Club Bilbao,athletic-club-bilbao,True,True,True,False,False,False
5,Real Sociedad de Fútbol,real-sociedad-donostia-san-sebastian,True,True,True,True,True,True
6,Valencia CF,valencia-valencia,True,True,True,False,False,False
7,Real Betis Balompié,real-betis-sevilla,True,True,True,True,True,True
8,UD Las Palmas,las-palmas-las-palmas-de-gran-canaria,True,False,True,False,False,False
9,Getafe Club de Fútbol,getafe-getafe-madrid,True,True,True,False,False,False


In [29]:
clubs_by_season_df['2021_22_la_liga'] = clubs_by_season_df['2021_22_la_liga'].astype(bool)
clubs_by_season_df['2022_23_la_liga'] = clubs_by_season_df['2022_23_la_liga'].astype(bool)
clubs_by_season_df['2023_24_la_liga'] = clubs_by_season_df['2023_24_la_liga'].astype(bool)
clubs_by_season_df['2021_22_uefa'] = clubs_by_season_df['2021_22_uefa'].astype(bool)
clubs_by_season_df['2022_23_uefa'] = clubs_by_season_df['2022_23_uefa'].astype(bool)
clubs_by_season_df['2023_24_uefa'] = clubs_by_season_df['2023_24_uefa'].astype(bool)
clubs_by_season_df.dtypes

team_name          object
team_slug          object
2021_22_la_liga      bool
2022_23_la_liga      bool
2023_24_la_liga      bool
2021_22_uefa         bool
2022_23_uefa         bool
2023_24_uefa         bool
dtype: object

In [30]:
# Droping the transactions of clubs not in la liga for a specific season (not yet promoted or relegated)


#Create a copy of the filtered_transactions_df to avoid overwriting it
la_liga_transactions_df = filtered_transactions_df.copy()

# Define the list of seasons

seasons = [
    {'name': '2021_22_la_liga', 'start_date': '2021-09-01', 'end_date': '2022-06-30'},
    {'name': '2022_23_la_liga', 'start_date': '2022-09-01', 'end_date': '2023-06-30'},
    {'name': '2023_24_la_liga', 'start_date': '2023-09-01', 'end_date': '2024-06-30'}
]

# Merge la_liga_transactions_df with clubs_by_season based on both club_slug_upon_transaction and team_slug
la_liga_transactions_df = pd.merge(
        la_liga_transactions_df,
        clubs_by_season_df,
        left_on=['club_slug_upon_transaction'],
        right_on=['team_slug'],
        how='inner'
    )

# Loop over each season
for season in seasons:
    # Filter clubs_out_season based on the current season and team slug
    season_column = season['name']
    clubs_out_season = clubs_by_season_df[clubs_by_season_df[season_column] == False]['team_slug']

    # Define the date range for the current season
    start_date = season['start_date']
    end_date = season['end_date']

    # Drop transactions within the date range and matching clubs_out_season
    la_liga_transactions_df.drop(
        la_liga_transactions_df[
            (la_liga_transactions_df['club_slug_upon_transaction'].isin(clubs_out_season)) &
            (la_liga_transactions_df['transaction_date'].between(start_date, end_date))
        ].index,
        inplace=True
    )

In [31]:
filtered_transactions_df.shape

(1349824, 38)

In [32]:
la_liga_transactions_df.shape

(1261014, 46)

In [33]:
la_liga_transactions_df.head()

,present_team_name,card_id,asset_id,card_name,season_year,card_slug,rarity,card_creation_date,grade,power,...,u23_until,club_slug_upon_transaction,team_name,team_slug,2021_22_la_liga,2022_23_la_liga,2023_24_la_liga,2021_22_uefa,2022_23_uefa,2023_24_uefa
0,Real Madrid CF,Card:00035848-b60d-4999-832c-e4dd8e939007,0x040004752c83837b25107500c71efa485aafb6cf1b82...,Nacho 2021-22 • Limited 218/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-218,limited,2021-11-25 03:40:39+00:00,7,1.045,...,NaN,real-madrid-madrid,Real Madrid CF,real-madrid-madrid,True,True,True,True,True,True
1,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31 08:58:22+00:00,13,1.075,...,NaN,real-madrid-madrid,Real Madrid CF,real-madrid-madrid,True,True,True,True,True,True
2,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31 08:58:22+00:00,13,1.075,...,NaN,real-madrid-madrid,Real Madrid CF,real-madrid-madrid,True,True,True,True,True,True
3,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31 08:58:22+00:00,13,1.075,...,NaN,real-madrid-madrid,Real Madrid CF,real-madrid-madrid,True,True,True,True,True,True
4,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31 08:58:22+00:00,13,1.075,...,NaN,real-madrid-madrid,Real Madrid CF,real-madrid-madrid,True,True,True,True,True,True


In [34]:
""" Used for checking drops magnitudes

# Filtered clubs
concerned_clubs = clubs_by_season_df[clubs_by_season_df['2023_24_la_liga'] == False]['team_slug']

# Filtered transactions per quarter
filtered_transactions_per_quarter = filtered_transactions_df[
    filtered_transactions_df['club_slug_upon_transaction'].isin(concerned_clubs)
].groupby(['transaction_date_year','transaction_date_quarter']).count()

# Display the transactions per quarter
filtered_transactions_per_quarter

"""

" Used for checking drops magnitudes\n\n# Filtered clubs\nconcerned_clubs = clubs_by_season_df[clubs_by_season_df['2023_24_la_liga'] == False]['team_slug']\n\n# Filtered transactions per quarter\nfiltered_transactions_per_quarter = filtered_transactions_df[\n    filtered_transactions_df['club_slug_upon_transaction'].isin(concerned_clubs)\n].groupby(['transaction_date_year','transaction_date_quarter']).count()\n\n# Display the transactions per quarter\nfiltered_transactions_per_quarter\n\n"

In [36]:
la_liga_transactions_df['u23_until'] = la_liga_transactions_df['u23_until'].fillna('NA')

In [37]:
#Removing "legends cards" 
la_liga_transactions_df = la_liga_transactions_df[la_liga_transactions_df['season_year'] > 2020]

In [38]:
la_liga_transactions_df.shape

(1246067, 46)

In [58]:
#la_liga_transactions_df.to_csv('la_liga_filtered_cards_transactions.csv', index=False)

#### f. Handling potential duplicates: 

In [39]:
la_liga_transactions_df.isna().sum()

present_team_name               0
card_id                         0
asset_id                        0
card_name                       0
season_year                     0
card_slug                       0
rarity                          0
card_creation_date              0
grade                           0
power                           0
xp                              0
serial_number                   0
shirt_number                    0
special_edition                 0
transaction_date                0
transaction_type                0
price_wei                       0
price_eur                       0
user_id                         0
user_slug                       0
player_slug                     0
card_creation_date_year         0
card_creation_date_quarter      0
card_creation_date_month        0
card_creation_date_week         0
card_creation_date_dayofyear    0
card_creation_date_season       0
transaction_date_year           0
transaction_date_quarter        0
transaction_da

In [40]:
la_liga_transactions_df.duplicated().sum()

0

In [41]:
# having not fetch transaction ID cannot assess if some are duplicated. Apparently not. 
la_liga_transactions_df.duplicated(subset=['card_id']).sum()

775803

#### Cleaning/preparing the Prices 

In [42]:
# Converting WEI in ETH (ETH prices are in WEI, which 1e-18 ETH)
la_liga_transactions_df['price_eth'] = la_liga_transactions_df['price_wei'] * 1e-18

##### categorisation of transactions
There are both paying and free transactions (REWARD,REFERRAL) as well as non usable paying transaction (cards bought in bundles or packs).
Also, in the analysis, it will be insightful to distinguish primary marked (english_auction and more recently intant_buy) and secondary_market (_OFFER). 

In [43]:
la_liga_transactions_df['transaction_type'].value_counts()

transaction_type
SINGLE_SALE_OFFER          450105
ENGLISH_AUCTION            316412
DIRECT_OFFER               212210
SINGLE_BUY_OFFER           132467
REWARD                     111393
INSTANT_BUY                 10775
REFERRAL                     9625
BUNDLED_ENGLISH_AUCTION      1693
PACK                         1301
TRANSFER                       58
MINT                           24
WITHDRAWAL                      2
DEPOSIT                         2
Name: count, dtype: int64

In [44]:
# Create the transaction_category column
conditions = [
    la_liga_transactions_df['transaction_type'].isin(['SINGLE_SALE_OFFER', 'ENGLISH_AUCTION', 'DIRECT_OFFER', 'SINGLE_BUY_OFFER', 'INSTANT_BUY']),
    la_liga_transactions_df['transaction_type'].isin(['REWARD', 'REFERRAL'])
]
values = ['paid_transaction', 'free_transaction']
default_value = 'other'
la_liga_transactions_df['transaction_category'] = np.select(conditions, values, default=default_value)

# Create the transaction_market column
market_conditions = [
    la_liga_transactions_df['transaction_type'].isin(['ENGLISH_AUCTION', 'INSTANT_BUY']),
    la_liga_transactions_df['transaction_type'].isin(['SINGLE_SALE_OFFER', 'DIRECT_OFFER', 'SINGLE_BUY_OFFER'])
]
market_values = ['Primary', 'Secondary']
market_default_value = 'not-applicable'
la_liga_transactions_df['transaction_market'] = np.select(market_conditions, market_values, default=market_default_value)


In [45]:
la_liga_transactions_df['transaction_category'].value_counts()

transaction_category
paid_transaction    1121969
free_transaction     121018
other                  3080
Name: count, dtype: int64

In [46]:
#Handle with the trade of cards (direct_offers that appears with price = 0)

# Rename the category from "paid_transaction" to "monetary_transaction"
la_liga_transactions_df['transaction_category'] = la_liga_transactions_df['transaction_category'].replace('paid_transaction', 'monetary_transaction')

# Create the new category "trade"
trade_condition = (la_liga_transactions_df['transaction_type'] == 'DIRECT_OFFER') & (la_liga_transactions_df['price_eur'] == 0)
la_liga_transactions_df.loc[trade_condition, 'transaction_category'] = 'trade'


In [47]:
la_liga_transactions_df['transaction_category'].value_counts()

transaction_category
monetary_transaction    928208
trade                   193761
free_transaction        121018
other                     3080
Name: count, dtype: int64

##### Flagging of abnormal transactions
In some cases (misprint or personal arrangements (loans, etc)) a out-of-market price will intervene. To flag in a very volatile market, we can calculate a rolling window statistic, such as the mean or median and flag the transactions that differs a lot from that. to be implemented if time 


#### i. Flagging current / precedent season card.
more and more elements in the gamemplay benefit more to the current season cards compared to the ones printed in the previous seasons. So the price impact may have evolve on that point. 
NB: A major gameplay change has intervene the 7th of February where it is a key element. Maybe it can be seen in the very last days of evolution but it will be a key if running an analysis in one month. 

In [48]:
la_liga_transactions_df.head(2)

,present_team_name,card_id,asset_id,card_name,season_year,card_slug,rarity,card_creation_date,grade,power,...,team_slug,2021_22_la_liga,2022_23_la_liga,2023_24_la_liga,2021_22_uefa,2022_23_uefa,2023_24_uefa,price_eth,transaction_category,transaction_market
0,Real Madrid CF,Card:00035848-b60d-4999-832c-e4dd8e939007,0x040004752c83837b25107500c71efa485aafb6cf1b82...,Nacho 2021-22 • Limited 218/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-218,limited,2021-11-25 03:40:39+00:00,7,1.045,...,real-madrid-madrid,True,True,True,True,True,True,0.0000,free_transaction,not-applicable
1,Real Madrid CF,Card:01058883-d9bc-4fe7-953e-1ac82197e006,0x040016f093c032839408074366792f3f16f388d20a20...,Nacho 2021-22 • Limited 488/1000,2021,jose-ignacio-fernandez-iglesias-2021-limited-488,limited,2022-05-31 08:58:22+00:00,13,1.075,...,real-madrid-madrid,True,True,True,True,True,True,0.0065,monetary_transaction,Primary


In [49]:
la_liga_transactions_df['in_season_card_transaction'] = la_liga_transactions_df['transaction_date_season'].astype(str).str.replace('season_', '') == la_liga_transactions_df['season_year'].astype(str)

In [50]:
la_liga_transactions_df['in_season_card_transaction'].value_counts()

in_season_card_transaction
True     895755
False    350312
Name: count, dtype: int64

#### Creation of Demand & Supply indicators: 

In [51]:
### For user balance: users with first buy & User with last sale 
### to see later if time


In [52]:
### Differenciation Manager/Trader

In [53]:
la_liga_transactions_df['user_slug'].nunique()

140915

In [54]:
top_100_user_transac_count = la_liga_transactions_df['user_slug'].value_counts().head(100).index
la_liga_transactions_df['user_category'] = np.where(la_liga_transactions_df['user_slug'].isin(top_100_user_transac_count), 'trader', 'manager')

In [55]:
#Creating a sub-dataset only on first transaction to ease some data analysis: 

la_liga_transactions_df = la_liga_transactions_df.sort_values('transaction_date')
first_la_liga_trans_df = la_liga_transactions_df.groupby('card_id').first().reset_index()
la_liga_cards_df = first_la_liga_trans_df[['card_id', 'asset_id', 'card_name', 'season_year', 'card_slug', 'rarity', 'card_creation_date', 'grade', 'power', 'xp', 'serial_number', 'shirt_number', 'special_edition', 'player_slug', 'card_creation_date_year', 'card_creation_date_month', 'card_creation_date_week', 'card_creation_date_dayofyear', 'card_creation_date_season', 'position', 'age', 'u23_eligible', 'u23_until', 'club_slug_upon_transaction']].copy()

In [56]:
la_liga_cards_df.shape

(470264, 24)

In [57]:
only_transactions_df = la_liga_transactions_df[['card_id', 'transaction_date', 'transaction_type', 'price_wei', 'price_eur', 'price_eth', 'user_id', 'user_slug', 'player_slug', 'transaction_date_year', 'transaction_date_month', 'transaction_date_week', 'transaction_date_dayofyear', 'transaction_date_season', 'club_slug_upon_transaction', 'transaction_category', 'transaction_market', 'in_season_card_transaction', 'user_category']].copy()

In [58]:
#la_liga_transactions_df.to_csv('la_liga_filtered_cards_transactions.csv', index=False)
#only_transactions_df.to_csv('transactions_only.csv',index=False)
#la_liga_cards_df.to_csv('la_liga_cards.csv',index=False)

#### Export to MySQL

In [59]:
pw = os.getenv('MySQLpwd')
connection_string = 'mysql+pymysql://root:' + pw + '@localhost:3306/'
engine = create_engine(connection_string)

In [60]:
#clubs_by_season_df.to_sql(name='clubs',con=engine, schema='final_project', if_exists='replace', index=False)

In [61]:
#la_liga_cards_df.to_sql(name='cards',con=engine, schema='final_project', if_exists='replace', index=False)

In [62]:
#only_transactions_df.dtypes

In [63]:
#only_transactions_df.to_sql(name='transactions',con=engine, schema='final_project', dtype={'transaction_date':sqlalchemy.DateTime()}, if_exists='replace', index=False)

In [64]:
players_club_hist_df['u23_until'] = players_club_hist_df['u23_until'].fillna('NA')

In [ ]:
cleaned_players_club_hist_df = players_club_hist_df[players_club_hist_df['club_slug'].isin(clubs_by_season_df['team_slug'])]

In [103]:
#cleaned_players_club_hist_df.to_sql(name='clubs_by_players',con=engine, schema='final_project', dtype={'contract_start_date':sqlalchemy.DateTime(),'contract_end_date':sqlalchemy.DateTime()},if_exists='replace', index=False)

NameError: name 'engine' is not defined

## Data Cleaning and Wrangling of the Players Scores Dataframe

In [66]:
players_scores_df = pd.read_csv('la_liga_players_scores.csv')

In [67]:
players_scores_df.shape

(121275, 13)

In [68]:
players_scores_df.isna().sum()

player_name                       0
player_slug                       0
player_id                         0
age                               0
u23_eligible                      0
u23_until                     93946
so5_score                         0
decisive_score                    0
gameweek_date                     0
gameweek_number                   0
gameweek_id                       0
rolling_avg_last_5_scores     44333
rolling_avg_last_15_scores    44333
dtype: int64

In [69]:
players_scores_df['u23_until'] = players_scores_df['u23_until'].fillna('NA')

In [70]:
players_scores_df['gameweek_date'] = pd.to_datetime(players_scores_df['gameweek_date'])

In [71]:
players_scores_df['decisive_score'].value_counts()

decisive_score
25.0     62228
35.0     43864
60.0     11035
15.0      2268
70.0      1501
80.0       232
5.0        105
90.0        28
0.0         10
100.0        4
Name: count, dtype: int64

In [72]:
# Problem of Quality of data -> Decisive Score is 25 when SO5 score, that is not normal. Handling: 
players_scores_df['decisive_score'] = np.where(players_scores_df['so5_score'] == 0, 0, players_scores_df['decisive_score'])


In [73]:
'''players_scores_df.to_sql(
    name='temp_players_scores',
    con=engine,
    schema='final_project',
    dtype={'gameweek_date': sqlalchemy.DateTime()},
    if_exists='replace',
    index=False
)'''

"players_scores_df.to_sql(\n    name='temp_players_scores',\n    con=engine,\n    schema='final_project',\n    dtype={'gameweek_date': sqlalchemy.DateTime()},\n    if_exists='replace',\n    index=False\n)"

In [74]:
players_scores_df = players_scores_df.drop(['rolling_avg_last_5_scores', 'rolling_avg_last_15_scores'], axis=1)

In [75]:

# Create "status_proxy" column to retrieve if a player was starter or sub: 
players_scores_df['status_proxy'] = np.where(players_scores_df['decisive_score'] > 30, 35,
                                            np.where((players_scores_df['decisive_score'] < 30) & (players_scores_df['decisive_score'] > 0), 25,
                                                     0))


In [76]:
#Creating the L5 and L15 metrics (rolling averages of scores when playing) + sum of status_proxy
# Sort the DataFrame by player_slug and gameweek_number
players_scores_df = players_scores_df.sort_values(['player_slug', 'gameweek_number'])

# Calculate the rolling average of the last 5 scores per player_slug
players_scores_df['L5'] = players_scores_df.groupby('player_slug')['so5_score'].rolling(window=5, min_periods=1).apply(lambda x: x[:-1].mean(), raw=False).reset_index(0, drop=True)

# Calculate the rolling average of the last 15 scores per player_slug
players_scores_df['L15'] = players_scores_df.groupby('player_slug')['so5_score'].rolling(window=15, min_periods=1).apply(lambda x: x[:-1].mean(), raw=False).reset_index(0, drop=True)

# Calculate the sum of status proxy to define then if a player is a regulare starter or regular sub: 
players_scores_df['Start15'] = players_scores_df.groupby('player_slug')['status_proxy'].rolling(window=15, min_periods=1).apply(lambda x: x[:-1].sum(), raw=False).reset_index(0, drop=True)

# Round L5 and L15 values to 2 decimal places
players_scores_df['L5'] = players_scores_df['L5'].round(2)
players_scores_df['L15'] = players_scores_df['L15'].round(2)

# Replace NaN values with 0 in L5 and L15 columns
players_scores_df['L5'] = players_scores_df['L5'].fillna(0)
players_scores_df['L15'] = players_scores_df['L15'].fillna(0)

# Affect the regular starting status based on the sum of last 15 Status Proxy
start_conditions = [players_scores_df['Start15'] > 400 , players_scores_df['Start15'] > 250]
values = ['regular_starter', 'regular_sub']
default_value = 'reserve'
players_scores_df['regular_status'] = np.select(start_conditions, values, default=default_value)



In [77]:
players_scores_df.head()

,player_name,player_slug,player_id,age,u23_eligible,u23_until,so5_score,decisive_score,gameweek_date,gameweek_number,gameweek_id,status_proxy,L5,L15,Start15,regular_status
52081,Aarón Escandell,aaron-escandell-banacloche,Player:2d383d99-a88d-4a95-8629-a27da90048b5,28,False,NA,41.9,35.0,2020-02-08 20:00:00+00:00,44,So5Fixture:8c9b6147-f176-492c-a413-936a39d23f8d,35,0.0,0.0,0.0,reserve
52080,Aarón Escandell,aaron-escandell-banacloche,Player:2d383d99-a88d-4a95-8629-a27da90048b5,28,False,NA,42.5,35.0,2020-06-28 17:30:00+00:00,76,So5Fixture:504b490c-e75c-4fe6-803b-952e35559818,35,41.9,41.9,35.0,reserve
52079,Aarón Escandell,aaron-escandell-banacloche,Player:2d383d99-a88d-4a95-8629-a27da90048b5,28,False,NA,69.2,60.0,2020-07-01 17:30:00+00:00,77,So5Fixture:aa043abd-86ff-4dc5-a170-6e2ba4e55bdd,35,42.2,42.2,70.0,reserve
52078,Aarón Escandell,aaron-escandell-banacloche,Player:2d383d99-a88d-4a95-8629-a27da90048b5,28,False,NA,0.0,0.0,2020-09-27 14:00:00+00:00,102,So5Fixture:947c14e3-a161-420d-924c-cb9972154ae9,0,51.2,51.2,105.0,reserve
52077,Aarón Escandell,aaron-escandell-banacloche,Player:2d383d99-a88d-4a95-8629-a27da90048b5,28,False,NA,0.0,0.0,2020-10-01 17:00:00+00:00,103,So5Fixture:04cb9f64-803a-4ea3-8516-2a163d5beebf,0,38.4,38.4,105.0,reserve


In [78]:
players_scores_df.isna().sum()

player_name        0
player_slug        0
player_id          0
age                0
u23_eligible       0
u23_until          0
so5_score          0
decisive_score     0
gameweek_date      0
gameweek_number    0
gameweek_id        0
status_proxy       0
L5                 0
L15                0
Start15            0
regular_status     0
dtype: int64

In [79]:
players_scores_df['regular_status'].value_counts()

regular_status
reserve            49423
regular_sub        38614
regular_starter    33238
Name: count, dtype: int64

In [80]:
# Filter the DataFrame for player_slug == 'memphis-depay'
memphis_scores = players_scores_df[players_scores_df['player_slug'] == 'memphis-depay']

# Sort the filtered DataFrame by gameweek_number
memphis_scores = memphis_scores.sort_values('gameweek_number')


In [81]:
memphis_scores[["player_name","so5_score","gameweek_number","L5","L15", "decisive_score", "status_proxy","status_proxy","Start15","regular_status"]].tail(50)

,player_name,so5_score,gameweek_number,L5,L15,decisive_score,status_proxy,status_proxy,Start15,regular_status
16838,Memphis Depay,60.0,345,11.75,17.11,60.0,35,35,205.0,reserve
16837,Memphis Depay,30.9,347,26.75,21.39,25.0,25,25,240.0,reserve
16836,Memphis Depay,0.0,349,28.52,23.60,0.0,0,0,265.0,regular_sub
16835,Memphis Depay,77.9,351,28.52,23.60,70.0,35,35,265.0,regular_sub
16834,Memphis Depay,38.9,353,42.20,29.16,35.0,35,35,300.0,regular_sub
16833,Memphis Depay,27.3,355,36.93,29.86,35.0,35,35,310.0,regular_sub
16831,Memphis Depay,82.4,357,36.03,30.38,60.0,35,35,320.0,regular_sub
16832,Memphis Depay,13.7,357,56.63,33.79,15.0,25,25,320.0,regular_sub
16830,Memphis Depay,0.0,359,40.58,29.76,0.0,0,0,310.0,regular_sub
16829,Memphis Depay,0.0,361,30.85,27.01,0.0,0,0,275.0,regular_sub


In [82]:
players_scores_df.shape

(121275, 16)

In [83]:
#export to CSV: 
players_scores_df.to_csv('cleaned_la_liga_players_scores.csv', index=False)


In [255]:
#export to SQL
"""
players_scores_df.to_sql(
    name='players_scores',
    con=engine,
    schema='final_project',
    dtype={'gameweek_date': sqlalchemy.DateTime()},
    if_exists='replace',
    index=False
)
"""

121275

In [84]:
players_scores_df.isna().sum()

player_name        0
player_slug        0
player_id          0
age                0
u23_eligible       0
u23_until          0
so5_score          0
decisive_score     0
gameweek_date      0
gameweek_number    0
gameweek_id        0
status_proxy       0
L5                 0
L15                0
Start15            0
regular_status     0
dtype: int64

In [94]:

distinct_gameweeks_df = players_scores_df[['gameweek_number', 'gameweek_date', 'gameweek_id']].drop_duplicates()
distinct_gameweeks_df = distinct_gameweeks_df.sort_values(by='gameweek_number')


In [95]:
distinct_gameweeks_df.tail(50)

,gameweek_number,gameweek_date,gameweek_id
59210,447,2024-02-05 19:30:00+00:00,So5Fixture:73a2e71f-84ad-49dc-aa3e-2105453584b1
90970,447,2024-02-03 17:15:00+00:00,So5Fixture:73a2e71f-84ad-49dc-aa3e-2105453584b1
63833,447,2024-02-04 15:15:00+00:00,So5Fixture:73a2e71f-84ad-49dc-aa3e-2105453584b1
3546,447,2024-02-04 20:00:00+00:00,So5Fixture:73a2e71f-84ad-49dc-aa3e-2105453584b1
50853,447,2024-02-04 14:00:00+00:00,So5Fixture:73a2e71f-84ad-49dc-aa3e-2105453584b1
1712,447,2024-02-03 20:30:00+00:00,So5Fixture:73a2e71f-84ad-49dc-aa3e-2105453584b1
84842,447,2024-02-04 01:10:00+00:00,So5Fixture:73a2e71f-84ad-49dc-aa3e-2105453584b1
51931,447,2024-02-03 15:15:00+00:00,So5Fixture:73a2e71f-84ad-49dc-aa3e-2105453584b1
92133,447,2024-02-04 12:00:00+00:00,So5Fixture:73a2e71f-84ad-49dc-aa3e-2105453584b1
32706,447,2024-02-04 16:05:00+00:00,So5Fixture:73a2e71f-84ad-49dc-aa3e-2105453584b1
